## Expert system implementation

In [0]:
import tempfile
import os
#!pip install git
#from git import Repo

# Check if pyswip package needs to be installed
try:
    from pyswip.prolog import Prolog
    from pyswip.easy import *
except ModuleNotFoundError as err:
    os.mkdir("PySWIP")
    Repo.clone_from("https://github.com/rohanshekhar/pyswip.git", "PySWIP/")
    os.symlink("PySWIP/pyswip/", "pyswip")
    from pyswip.prolog import Prolog
    from pyswip.easy import *
    
import numpy as np
prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

In [0]:

KB = """
% RULES
% activities that will be recommended
activity(bodyweight) :- goal(build_muscle), (impediment(no); not(impediment(arm))).
activity(running):- goal(lose_fat), not(place(indoor)), (impediment(no); not(impediment(knee))), not(weather(rainy)).
activity(hiking):- goal(stay_active), not(place(indoor)), (impediment(no); not(impediment(leg))),  not(weather(rainy)).
activity(swimming):- goal(build_muscle), place(indoor), (impediment(no); not(impediment(arm))).
activity(cycling):- goal(lose_fat),not(place(indoor)), (impediment(no); not(impediment(leg))),  not(weather(rainy)).
activity(yoga):- goal(stretch), not(place(indoor)), (impediment(no); not(impediment(leg))), not(weather(rainy)).
activity(yoga):- goal(stretch), place(indoor), (impediment(no); not(impediment(leg))).
activity(walking):- goal(stay_active), not(place(indoor)), (impediment(no); not(impediment(leg))),  not(weather(rainy)).
% if person cannot do other activity
activity(personal_trainer).

% recommendatios themselves, either in group or on your own
suggest('run with Rise + Run with Morgan at Salesforce'):- activity(running),not(group(no)), distance(short).
suggest('run with San Francisco Friendly Run Club at Dolores Park'):- activity(running),  not(group(no)), distance(medium).
suggest('run with Marin Running Club'):- activity(running), not(group(no)), distance(long).
suggest('run on your own'):- activity(running), group(no).

suggest('hike with Urban and Bay Hikers'):- activity(hiking), not(group(no)), distance(short),  free(no).
suggest('hike with Golden Gate Hiking Group '):-  activity(hiking), not(group(no)), distance(medium), activity(hiking).
suggest("hike with group Let's go Chasing Waterfalls"):- activity(hiking), not(group(no)), distance(long).
suggest('hike on your own'):- activity(hiking), group(no).

suggest('do bodyweight exercises with Overcoming Bodyweight Strength Training at Marina Fitness Court'):- activity(bodyweight), not(group(no)), distance(short).
suggest('do bodyweight exercises Outdoor Natural Movement Tribe in Berkeley'):- activity(bodyweight), not(group(no)),distance(long).
suggest('do bodyweight exercises Free Bootcamp at Golden Gate'):- activity(bodyweight), not(group(no)),distance(medium).
suggest('do bodyweight on your own'):- activity(bodyweight), group(no).

suggest('swim with Water World Swim at Aquatic Park'):- activity(swimming), not(group(no)), distance(medium).
suggest('swim on your own'):- activity(swimming), group(no).

suggest('cycle with SF Bike Kitchen in Mission'):- activity(cycling), not(group(no)),  distance(short).
suggest('cycle with Bi-cyling at Golden Gate'):- activity(cycling),not(group(no)), distance(medium).
suggest('cycle with Alameda Cycling Group'):- activity(cycling),  not(group(no)), distance(long).
suggest('cycle on your own'):- activity(cycling), group(no).

suggest('walk with Weekend Walking Adventure'):- activity(waliking), not(group(no)), distance(short).
suggest('walk with SF Stairway Photowalks'):- activity(waliking), not(group(no)), distance(medium).
suggest('walk with Walking Meditation Group'):- activity(waliking), not(group(no)), distance(long).
suggest('walk on your own'):- activity(waliking), group(no).

suggest('do yoga with Free Yoga SF in Mission'):- activity(yoga), not(group(no)), distance(short).
suggest('do yoga with Yoga in the Presidio'):- activity(yoga), not(group(no)), distance(medium).
suggest('do yoga with Easy Bay Outdoor Yoga Group'):- activity(yoga), not(group(no)), distance(long).
suggest('do yoga on your own'):- activity(yoga), group(no).

suggest('work with a personal trainer'):- activity(personal_trainer).

% COVID related:
group(no):- vaccinated(no), underlying_condition(yes).
underlying_condition(yes):- condition(diabetes); condition(heart_problems) ; condition(overweight).

% askables
goal(X):- ask(goal,X).
place(X) :- ask(place,X).
weather(X) :- ask(weather, X).
impediment(X) :- ask(impediment, X).
vaccinated(X) :- ask(vaccinated,X).
condition(X) :- ask(condition, X).
distance(X):- ask(distance, X).
free(X):- ask(free, X).

% Asking clauses
%multivalued(impediment).
multivalued(goal).
%multivalued(condition).
%multivalued(distance).

ask(A, V):-
known(yes, A, V), % succeed if true
!. % stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

% If not multivalued, and already known, don't ask again for a different value.
ask(A, V):-
\+multivalued(A),
known(yes, A, V2),
V \== V2,
!.

ask(A, V):-
read_py(A,V,Y), % get the answer
asserta(known(Y, A, V)), % remember it
write_py(known(Y, A, V)),
Y == yes. % succeed or fail

"""

In [0]:
def write_py(X):
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    if isinstance(Y, Variable):
        
        if str(A) == "goal":
            print("Is your goal to {}?".format(str(V)))
        elif str(A) == "place":
            print("Do you want to be inside?")
        elif str(A) == "weather":
            print("Is it {}?".format(str(V)))
        elif str(A) == "impediment":
            print("Do you have {} impediments?".format(str(V)))
        elif str(A) == "vaccinated":
            print("Are you unvaccinated?")
        elif str(A) == "condition":
            print("Do you have the condition {}?".format(str(V)))
        elif str(A) == "distance":
            print("Do you want to travel a {} distance?".format(str(V)))
        elif str(A) == "free":
            print("Should it be free?")
            
        print()
        print("1. Yes")
        print("2. No")

        res = input("Answer: ")
        if res in ["1", "yes", "Yes", "YES"]:
            Y.unify("yes")
        else:
            Y.unify("no")
        return True
    else:
        return False

write_py.arity = 1
read_py.arity = 3

registerForeign(read_py)
registerForeign(write_py)

# Create a temporary file with the KB in it
(FD, name) = tempfile.mkstemp(suffix='.pl', text = "True")
with os.fdopen(FD, "w") as text_file:
    text_file.write(KB)
prolog.consult(name) # open the KB for consulting
os.unlink(name) # Remove the temporary file

call(retractall(known))
suggest = [s for s in prolog.query("suggest(X).", maxresult=1)]
print("We suggest you to " + (suggest[0]['X'] + "." if suggest else "unknown."))

## Attempt at menu-based responses

In [0]:
KB = """
% RULES
% activities that will be recommended
activity(bodyweight) :- goal(build_muscle), not(impediment(arm)).
activity(running):- goal(lose_fat), not(place(indoor)), not(impediment(knee)), not(weather(rainy)).
activity(hiking):- goal(stay_active), not(place(indoor)), not(impediment(leg)),  not(weather(rainy)).
activity(swimming):- goal(build_muscle),place(indoor), not(impediment(arm)).
activity(cycling):- goal(lose_fat),not(place(indoor)),  not(impediment(leg)),  not(weather(rainy)).
activity(yoga):- goal(stretch), not(place(indoor)),  not(impediment(leg)),  not(weather(rainy)).
activity(yoga):- goal(stretch), place(indoor), not(impediment(leg)). 
activity(walking):- goal(stay_active), not(place(indoor)), not(impediment(leg)),  not(weather(rainy)).
% if person cannot do other activity
activity(personal_trainer).

%distance(short) :- not(distance(medium)), not(distance(long)).
%distance(medium) :- not(distance(short)), not(distance(long)).
%distance(long) :- not(distance(short)), not(distance(medium)).

% recommendatios themselves, either in group or on your own
suggest('run with Rise + Run with Morgan at Salesforce'):- activity(running),not(group(no)), distance(short).
suggest('run with San Francisco Friendly Run Club at Dolores Park'):- activity(running),  not(group(no)), distance(medium).
suggest('run with Marin Running Club'):- activity(running), not(group(no)), distance(long).
suggest('run on your own'):- activity(running), group(no).

suggest('hike with Urban and Bay Hikers'):- activity(hiking), not(group(no)), distance(short),  free(no).
suggest('hike with Golden Gate Hiking Group '):-  activity(hiking), not(group(no)), distance(medium), activity(hiking).
suggest("hike with group Let's go Chasing Waterfalls"):- activity(hiking), not(group(no)), distance(long).
suggest('hike on your own'):- activity(hiking), group(no).

suggest('do bodyweight exercises with Overcoming Bodyweight Strength Training at Marina Fitness Court'):- activity(bodyweight), not(group(no)), distance(short).
suggest('do bodyweight exercises Outdoor Natural Movement Tribe in Berkeley'):- activity(bodyweight), not(group(no)),distance(long).
suggest('do bodyweight exercises Free Bootcamp at Golden Gate'):- activity(bodyweight), not(group(no)),distance(medium).
suggest('do bodyweight on your own'):- activity(bodyweight), group(no).

suggest('swim with Water World Swim at Aquatic Park'):- activity(swimming), not(group(no)), distance(medium).
suggest('swim on your own'):- activity(swimming), group(no).

suggest('cycle with SF Bike Kitchen in Mission'):- activity(cycling), not(group(no)),  distance(short).
suggest('cycle with Bi-cyling at Golden Gate'):- activity(cycling),not(group(no)), distance(medium).
suggest('cycle with Alameda Cycling Group'):- activity(cycling),  not(group(no)), distance(long).
suggest('cycle on your own'):-  activity(cycling), group(no).

suggest('walk with Weekend Walking Adventure'):- activity(waliking), not(group(no)), distance(short).
suggest('walk with SF Stairway Photowalks'):- activity(waliking), not(group(no)), distance(medium).
suggest('walk with Walking Meditation Group'):- activity(waliking), not(group(no)), distance(long).
suggest('walk on your own'):-   activity(waliking), group(no).

suggest('do yoga with Free Yoga SF in Mission'):- activity(yoga), not(group(no)), distance(short).
suggest('do yoga with Yoga in the Presidio'):- activity(yoga), not(group(no)), distance(medium).
suggest('do yoga with Easy Bay Outdoor Yoga Group'):- activity(yoga), not(group(no)), distance(long).
suggest('do yoga on your own'):-  activity(yoga), group(no).

suggest('work with a personal trainer'):- activity(personal_trainer).

% COVID related:
group(no):- vaccinated(no), underlying_condition(yes).
underlying_condition(yes):- condition(diabetes); condition(heart_problems) ; condition(overweight).

% askables
goal(X):- ask(goal,X).
place(X) :- ask(place,X).
weather(X) :- ask(weather, X).
impediment(X) :- ask(impediment, X).
vaccinated(X) :- ask(vaccinated,X).
condition(X) :- ask(condition, X).
distance(X):- ask(distance, X).
free(X):- ask(free, X).

% Asking clauses
%multivalued(impediment).
multivalued(goal).
%multivalued(condition).
%multivalued(distance).

ask(A, V):-
known(yes, A, V), % succeed if true
!. % stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

% If not multivalued, and already known, don't ask again for a different value.
ask(A, V):-
\+multivalued(A),
known(yes, A, V2),
V \== V2,
!.

ask(A, V):-
\+multivalued(A)
read_py(A,V,Y), % get the answer
asserta(known(Y, A, V)), % remember it
write_py(known(Y, A, V)),
Y == yes. % succeed or fail

ask(A, V):-
multivalued(A),
read_py2(A,V,Y,L), % get the answer
test(L,Ln, I),
store(L, I),
asserta(known(Y, A, V)), % remember it
write_py(known(Y, A, V)),
Y == yes. % succeed or fail


store([]).
store(X, [H|T]) :- test(H),  asserta(known(yes, A, H)),write_py(known(yes, A, H)),store(T).
"""

In [0]:
import ast
def test(L , Ln, I):
    list(L)
    a = L.pop()
    I.unify(a)
    Ln.unify(L)
    L.unify(L)
    

def write_py(X):
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    if isinstance(Y, Variable):
        response = input(str(A) + " is " + str(V) + "? ")
        Y.unify(response)
        return True
    else:
        return False

def read_py2(A,V,Y, L):

    if isinstance(Y, Variable):
        if str(A) == "goal":
            goal_list = ["build_muscle", "lose_fat", "stay_active", "stretch"]
            print("What are your goals? ")
            for i in range(len(goal_list)):
                print("{}. {}".format(str(i+1), goal_list[i]))
            res = input("Answer (input list of numbers): ")
            res = ast.literal_eval(res)
            sel_goals = []
            x = 0
            for i in res:
                if goal_list[i-1] == str(V):
                    Y.unify("yes")
                    #print("yes")
                    #print(Y)
                    x = 1
                else:
                    sel_goals.append(goal_list[i-1])
            L.unify(str(sel_goals).replace("'", ''))
            #print(str(sel_goals).replace("'", ''))
            
            if x == 1:
                Y.unify("no")
                
                           
            
        #response = input(str(A) + " is " + str(V) + "? ")
        #Y.unify(response)
        return True
    else:
        return False

test.arity = 1
write_py.arity = 1
read_py.arity = 3
read_py2.arity = 4

registerForeign(read_py)
registerForeign(read_py2)
registerForeign(write_py)
registerForeign(test)

# Create a temporary file with the KB in it
(FD, name) = tempfile.mkstemp(suffix='.pl', text = "True")
with os.fdopen(FD, "w") as text_file:
    text_file.write(KB)
prolog.consult(name) # open the KB for consulting
os.unlink(name) # Remove the temporary file

call(retractall(known))
suggest = [s for s in prolog.query("suggest(X).", maxresult=1)]

print("We suggest you to " + (suggest[0]['X'] + "." if suggest else "unknown."))